In [7]:
from database.adatabase import ADatabase
from extractor.alp_client_extractor import ALPClientExtractor
from extractor.tiingo_extractor import TiingoExtractor
from processor.processor import Processor as processor
from datetime import datetime, timedelta
from tqdm import tqdm 
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
from time import sleep
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import argrelextrema


In [8]:
# Initialize parameters
window = 300
gain_window = 60
extractor = ALPClientExtractor(os.getenv("APCAKEY"), os.getenv("APCASECRET"))
tickers = ["AAVE", "AVAX", "BAT", "BTC", "CRV", "DOGE", "DOT", "ETH", "GRT", "LINK", "LTC", "MKR", "SHIB", "SUSHI", "UNI", "XTZ", "YFI"]
tickers = [x + "/USD" for x in tickers]

prices = []
start = datetime.now() - timedelta(days=1)
end = datetime.now()

# Loop through each ticker to fetch prices and calculate metrics
for ticker in tickers:
    response = extractor.crypto_interval(ticker, start)
    price = pd.DataFrame(response["bars"][ticker]).rename(columns={"c": "adjclose", "t": "date"})[["date", "adjclose"]]
    
    # Convert date and sort
    price["date"] = pd.to_datetime(price["date"])
    price = price.sort_values("date")
    
    # Calculate metrics
    price["rolling_return"] = ((price["adjclose"].rolling(window).mean() - price["adjclose"]) / price["adjclose"])
    price["bollinger"] = ((price["adjclose"].rolling(window).mean() - price["adjclose"].rolling(window).std() - price["adjclose"]) / price["adjclose"])
    price["return"] = price["adjclose"].pct_change(gain_window)
    price["avg_return"] = price["return"].pct_change(gain_window).rolling(window).mean()
    price["coev"] = (price["adjclose"].rolling(window).mean() / price["adjclose"].rolling(window).std())
    price["ticker"] = ticker
    prices.append(price)

# Combine all prices into a single DataFrame
sim = pd.concat(prices).sort_values("date").dropna()

In [9]:
metrics = [
    "rolling_return",
    "bollinger",
    "return",
    "avg_return",
    "coev"
]

In [10]:
import copy
analysis = []
date_range = list(sim["date"].unique())
base_position = {"date":date_range[0],"ticker":"","quantity":0,"adjclose":0,"buy_price":0,"buy_date":date_range[0]}
for metric in metrics:
    for ascending in [True,False]:
        cash = 100
        position = copy.deepcopy(base_position)
        cashes = []
        for date in tqdm(date_range):
            try:
                position = copy.deepcopy(position)
                position["date"] = date
                today = sim[sim["date"]==date]
                if today.index.size >= 0:
                    opportunity = today.sort_values(metric,ascending=ascending).iloc[0]
                    if position["ticker"] != "":
                        position["adjclose"] = today[today["ticker"]==position["ticker"]]["adjclose"].iloc[0].item()
                        pnl_req = ((position["adjclose"] - position["buy_price"]) / position["buy_price"] >= 0.005)
                        loss_req = ((position["adjclose"] - position["buy_price"]) / position["buy_price"] <= -0.005)
                        hpr_req = (date - position["buy_date"]).days > float(5/(24*60))
                        if pnl_req or loss_req or hpr_req:
                            try:
                                cash = position["adjclose"] * position["quantity"]
                                cashes.append(cash)
                                position = copy.deepcopy(base_position)
                                position["date"] = date
                                position["ticker"] = opportunity["ticker"]
                                position["quantity"] = cash / opportunity["adjclose"] 
                                position["adjclose"] = opportunity["adjclose"]
                                position["buy_price"] = opportunity["adjclose"]
                                position["buy_date"] = date
                            except Exception as e:
                                continue
                    else:
                        position["date"] = date
                        position["ticker"] = opportunity["ticker"]
                        position["quantity"] = cash / opportunity["adjclose"] 
                        position["adjclose"] = opportunity["adjclose"]
                        position["buy_price"] = opportunity["adjclose"]
                        position["buy_date"] = date
            except Exception as e:
                continue
        position["metric"] = metric
        position["ascending"] = ascending
        position["pv"] = position["adjclose"] * position["quantity"]
        analysis.append(copy.deepcopy(position))

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1835/1835 [00:06<00:00, 286.88it/s]


In [11]:
a = pd.DataFrame(analysis)
a.sort_values("pv",ascending=False)

,date,ticker,quantity,adjclose,buy_price,buy_date,metric,ascending,pv
1,2024-09-27 14:29:00+00:00,BAT/USD,532.824490,0.197295,0.197115,2024-09-27 14:24:00+00:00,rolling_return,False,105.123608
4,2024-09-27 14:29:00+00:00,UNI/USD,14.144040,7.387600,7.364140,2024-09-27 06:56:00+00:00,return,True,104.490509
9,2024-09-27 14:29:00+00:00,BTC/USD,0.001568,66345.435000,66117.710000,2024-09-27 14:11:00+00:00,coev,False,104.059018
8,2024-09-27 14:29:00+00:00,YFI/USD,0.018721,5463.662500,5447.820000,2024-09-27 14:01:00+00:00,coev,True,102.283775
7,2024-09-27 14:29:00+00:00,GRT/USD,537.267763,0.189315,0.188910,2024-09-27 08:37:00+00:00,avg_return,False,101.712847
6,2024-09-27 14:29:00+00:00,YFI/USD,0.018610,5463.662500,5465.500000,2024-09-27 14:23:00+00:00,avg_return,True,101.680912
2,2024-09-27 14:29:00+00:00,AAVE/USD,0.586667,170.950000,171.015000,2024-09-27 14:25:00+00:00,bollinger,True,100.290774
5,2024-09-27 14:29:00+00:00,LTC/USD,1.415502,70.408500,70.195850,2024-09-27 12:32:00+00:00,return,False,99.663406
3,2024-09-27 14:29:00+00:00,BAT/USD,502.478043,0.197295,0.196754,2024-09-27 14:22:00+00:00,bollinger,False,99.136405
0,2024-09-27 14:29:00+00:00,CRV/USD,310.730588,0.317580,0.317064,2024-09-27 14:25:00+00:00,rolling_return,True,98.681665
